## Difference Project

Some useful utility function.

In [1]:
function SoftThreshold(z, lambda)
    abs(z) < lambda ? 0. : z > 0 ? z - lambda : z + lambda
end

@show SoftThreshold(3, 1)
@show SoftThreshold(-3, 1)
@show SoftThreshold(-0.5, 1);

SoftThreshold(3,1) = 2
SoftThreshold(-3,1) = -2
SoftThreshold(-0.5,1) = 0.0


Generate data and compute precision matrices

In [35]:
p = 200
Sigmax = eye(p,p)
Sigmay = zeros(p,p)
rho = 0.7
for i=1:p
    for j=1:p
        Sigmay[i,j]=rho^abs(i-j)
    end
end

sqmy = sqrtm(Sigmay)

n = 1000
X = randn(n,p)
Y = randn(n,p) * sqmy;

hSx = cov(X)
hSy = cov(Y);

In [15]:
B = inv(Sigmax) - inv(Sigmay)
B[1:5,1:5]

5×5 Array{Float64,2}:
 -0.960784      1.37255      -8.59879e-17  -8.59879e-17   7.61918e-17
  1.37255      -1.92157       1.37255       1.52384e-16  -2.86263e-16
 -3.85334e-16   1.37255      -1.92157       1.37255       4.00551e-16
 -5.48809e-17   1.99154e-16   1.37255      -1.92157       1.37255    
  1.28117e-16  -2.21414e-16   3.65688e-16   1.37255      -1.92157    

Solve the problem using Convex and Mosek for comparison purposes

In [18]:
using Convex
using Mosek

solver = MosekSolver(LOG=1)
set_default_solver(solver);

In [19]:
lambda = 0.2

Delta = Variable(p,p);
prob = minimize(quadform(vec(Delta), kron(hSy,hSx)) / 2 - trace((hSy-hSx)*Delta) +  lambda * norm(vec(Delta), 1))
prob.constraints += [Delta == Delta']
@time solve!(prob)

sol = Delta.value

Computer
  Platform               : Linux/64-X86    

Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 1606            
  Cones                  : 2               
  Scalar variables       : 1207            
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Conic interior-point optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Total number of eliminations : 401
Eliminator terminated.
Eliminator started.
Total number of eliminations : 401
Eliminator terminated.
Eliminator - tries                  : 2                 time                   : 0.00            
Eliminator - elim's                 : 401             
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep. 

20×20 Array{Float64,2}:
 -0.194094     0.656217     3.68185e-8  …   6.26885e-9    1.0917e-8 
  0.656217    -0.586282     0.671745        6.88177e-9    7.42906e-9
  3.68191e-8   0.671745    -0.695651        9.68325e-9    2.51778e-8
  5.24489e-8   1.42774e-7   0.758148        1.32818e-8    1.97006e-8
  1.2774e-8    1.98271e-8   2.73477e-8      2.02056e-8    1.59234e-8
  1.54955e-8   6.50583e-8   2.85802e-8  …   1.45964e-8    1.30684e-8
  2.80326e-8   5.0776e-8    3.18651e-8      5.02339e-10   1.87989e-9
  1.67787e-8   9.30505e-8   2.11904e-8      5.3961e-9     2.33064e-8
  3.83328e-8   3.87108e-8   3.05129e-8     -2.30296e-8    1.56644e-8
  1.81474e-8   2.40762e-8   2.6495e-8       9.96472e-9    1.72882e-8
  1.76739e-8   3.66469e-8   1.92273e-8  …   1.49561e-8    1.20307e-8
  1.77859e-8   1.95918e-8   3.19555e-8      2.14175e-8    1.30568e-8
  1.2779e-8    1.57516e-8   1.8672e-8       3.52469e-8    7.4844e-9 
  5.17417e-9   1.8021e-8    2.18106e-8      1.80258e-8    1.82884e-8
 -1.12141e

Shooting algorithm for the objective

In [20]:
function DiffCovShooting(hSx, hSy, lambda, Ups; maxIter=2000, optTol=1e-7)
    p = size(hSx, 1)
        
    Delta = zeros(p, p)
    A = zeros(p, p)
    
    iter = 1;
    while iter < maxIter    
        if mod(iter, 10) == 0
            @show "outer $iter"
        end
    
        fDone = true
        for a=1:p
            for b=a:p
                if a==b
                    # diagonal elements
                    x0 = hSx[a,a]*hSy[a,a]/2
                    x1 = A[a,a] - hSy[a,a] + hSx[a,a]
                    tmp = SoftThreshold(-x1/x0/2 + Delta[a,b], lambda*Ups[a,b] / 2 / x0)
                else
                    # off-diagonal elements
                    x0 = (hSx[a,a]*hSy[b,b] + hSx[b,b]*hSy[a,a])/2 + hSx[a,b]*hSy[a,b]
                    x1 = A[a,b] + A[b,a] - 2*(hSy[a,b] - hSx[a,b])
                    tmp = SoftThreshold(-x1/x0/2 + Delta[a,b], lambda*Ups[a,b] / x0)
                end
                
                h = tmp - Delta[a,b]
                Delta[a,b] = tmp
                Delta[b,a] = tmp
                if abs(h) > optTol
                    fDone = false
                end
                for j=1:p
                    for k=1:p
                        if a == b
                            A[j,k] = A[j,k] + h * hSx[j,a]*hSy[a,k]
                        else                            
                            A[j,k] = A[j,k] + h * (hSx[j,a]*hSy[b,k] + hSx[j,b]*hSy[a,k])
                        end
                    end
                end
            end
        end
        
        iter = iter + 1;
        if fDone
            break
        end
    end
    Delta
end

DiffCovShooting (generic function with 1 method)

In [36]:
lambda = 0.2
@time solShoot = DiffCovShooting(hSx, hSy, lambda, ones(p,p))

"outer $(iter)" = "outer 10"
"outer $(iter)" = "outer 20"
"outer $(iter)" = "outer 30"
241.470222 seconds (674 allocations: 1012.250 KB)


200×200 Array{Float64,2}:
 -0.158825   0.596554   0.0       …   0.0        0.0        0.0     
  0.596554  -0.848799   0.648284      0.0        0.0        0.0     
  0.0        0.648284  -0.876228      0.0        0.0        0.0     
  0.0        0.0        0.721084      0.0        0.0        0.0     
  0.0        0.0        0.0           0.0        0.0        0.0     
  0.0        0.0        0.0       …   0.0        0.0        0.0     
  0.0        0.0        0.0           0.0        0.0        0.0     
  0.0        0.0        0.0           0.0        0.0        0.0     
  0.0        0.0        0.0           0.0        0.0        0.0     
  0.0        0.0        0.0           0.0        0.0        0.0     
  0.0        0.0        0.0       …   0.0        0.0        0.0     
  0.0        0.0        0.0           0.0        0.0        0.0     
  0.0        0.0        0.0           0.0        0.0        0.0     
  ⋮                               ⋱                                 
  0.0   

In [22]:
solShoot[1:7,1:7]

7×7 Array{Float64,2}:
 -0.194102   0.656222   0.0        0.0        0.0        0.0        0.0     
  0.656222  -0.586286   0.671748   0.0        0.0        0.0        0.0     
  0.0        0.671748  -0.695654   0.758151   0.0        0.0        0.0     
  0.0        0.0        0.758151  -0.885963   0.588594   0.0        0.0     
  0.0        0.0        0.0        0.588594  -0.630712   0.633719   0.0     
  0.0        0.0        0.0        0.0        0.633719  -0.767458   0.735599
  0.0        0.0        0.0        0.0        0.0        0.735599  -0.718711

In [23]:
@show dot(vec(sol), kron(hSy,hSx)*vec(sol)) / 2 - trace((hSy-hSx)*sol) +  lambda * norm(vec(sol), 1)
@show dot(vec(solShoot), kron(hSy,hSx)*vec(solShoot)) / 2 - trace((hSy-hSx)*solShoot) +  lambda * norm(vec(solShoot), 1);

(dot(vec(sol),kron(hSy,hSx) * vec(sol)) / 2 - trace((hSy - hSx) * sol)) + lambda * norm(vec(sol),1) = -5.599742591346342
(dot(vec(solShoot),kron(hSy,hSx) * vec(solShoot)) / 2 - trace((hSy - hSx) * solShoot)) + lambda * norm(vec(solShoot),1) = -5.5997435183754085


Implementation of the active shooting algorithm

In [24]:
function updateDelta!(Delta, hSx, hSy, A, lambda; maxIter=2000, optTol=1e-7)    
    # Delta is a sparse matrix stored in CSC format    
    # only diagonal and lower triangular elements are used
    
    # extract fields from sparse matrix Delta 
    colptr = Delta.colptr
    nzval = Delta.nzval
    rowval = Delta.rowval
    
    p = size(Delta, 1)
    
    iter = 1
    while iter <= maxIter
       if mod(iter, 100) == 0
            @show "inner $iter"
        end
        
        fDone = true
        for colInd=1:p
            for j=colptr[colInd]:colptr[colInd+1]-1
                rowInd = rowval[j]
                
                # a sanity check in 
                if rowInd < colInd
                    continue
                end
                
                if rowInd==colInd
                    # diagonal elements
                    x0 = hSx[rowInd,rowInd]*hSy[rowInd,rowInd]/2
                    x1 = A[rowInd,rowInd] - hSy[rowInd,rowInd] + hSx[rowInd,rowInd]
                    tmp = SoftThreshold(-x1/x0/2 + nzval[j], lambda[rowInd,colInd] / 2 / x0)
                else
                    # off-diagonal elements
                    x0 = (hSx[rowInd,rowInd]*hSy[colInd,colInd] + hSx[colInd,colInd]*hSy[rowInd,rowInd])/2 
                              + hSx[rowInd,colInd]*hSy[rowInd,colInd]
                    x1 = A[rowInd,colInd] + A[colInd,rowInd] - 2*(hSy[rowInd,colInd] - hSx[rowInd,colInd])
                    tmp = SoftThreshold(-x1/x0/2 + nzval[j], lambda[rowInd,colInd] / x0)
                end
                
                # size of update
                h = tmp - nzval[j]
                # update is too big -- not done
                if abs(h) > optTol
                    fDone = false
                end                
                # update Delta
                nzval[j] = tmp   
                
                # update A -- only active set
                for ci=1:p
                    for k=colptr[ci]:colptr[ci+1]-1
                        ri = rowval[k]
                        
                        if rowInd == colInd                                     
                            A[ri,ci] = A[ri,ci] + h * hSx[ri,rowInd]*hSy[rowInd,ci]
                            if ri != ci
                                A[ci,ri] = A[ci,ri] + h * hSx[ci,rowInd]*hSy[rowInd,ri]
                            end
                        else
                            A[ri,ci] = A[ri,ci] + h * (hSx[ri,colInd]*hSy[rowInd,ci] + hSx[ri,rowInd]*hSy[colInd,ci])
                            if ri != ci
                                A[ci,ri] = A[ci,ri] + h * (hSx[ci,colInd]*hSy[rowInd,ri] + hSx[ci,rowInd]*hSy[colInd,ri])
                            end                            
                        end
                    end
                end   # end update A                
            end
        end            
        
        iter = iter + 1;
        if fDone
            break
        end
        
    end  # while
    
    sparse(Delta)
end

function findViolator!(active_set, Delta, A, hSx, hSy, lambda; kktTol=1e-7)   
    p = size(hSx, 1)
    
    tmp = abs((A + A') / 2 - (hSy - hSx)) - lambda
    ind = indmax(tmp) 
    if tmp[ind] > kktTol
        push!(active_set, ind)
        Delta[ind] = eps()
    else
        ind = 0
    end
    
    return ind    
end

function DiffCovActiveShooting(hSx, hSy, lambda; maxIter=1000, maxInnerIter=1000, optTol=1e-7, Delta = [])
    p = size(hSx, 1)
        
    if isempty(Delta)
        Delta = spzeros(p, p)
        A = zeros(p, p)
        
        active_set = Array(Integer, 0)        
        indAdd = findViolator!(active_set, Delta, A, hSx, hSy, lambda)    
        if indAdd == 0
            return Delta
        end
    else
        lDelta = tril(Delta, -1)
        dDelta = spdiagm(diag(Delta))
        A = hSx * (lDelta + lDelta' + dDelta) * hSy
        
        # make sure Delta is lower triangular
        Delta = lDelta + dDelta
        active_set = find(Delta)
    end     
    
    
    
    iter = 1;
    while iter < maxIter    
        if mod(iter, 100) == 0
            @show "outer $iter"
        end
    
        old_active_set = copy(active_set)
        updateDelta!(Delta, hSx, hSy, A, lambda; maxIter=maxInnerIter, optTol=optTol)
        active_set = find(Delta)
        
        # update A
        lDelta = tril(Delta, -1)
        dDelta = spdiagm(diag(Delta))
        A = hSx * (lDelta + lDelta' + dDelta) * hSy
        # add violating element into active set 
        indAdd = findViolator!(active_set, Delta, A, hSx, hSy, lambda)         
        
        if old_active_set == active_set
            break
        else
        end       
        
        iter = iter + 1;
        
    end
    Delta
end

DiffCovActiveShooting (generic function with 1 method)

In [37]:
@time solShootFast = DiffCovActiveShooting(hSx, hSy, lambda*ones(p,p), optTol=1e-7)

"outer $(iter)" = "outer 100"
"outer $(iter)" = "outer 200"
"outer $(iter)" = "outer 300"
"outer $(iter)" = "outer 400"
 22.403817 seconds (61.38 k allocations: 1.177 GB, 0.27% gc time)


200×200 sparse matrix with 427 Float64 nonzero entries:
	[1  ,   1]  =  -0.158825
	[2  ,   1]  =  0.596554
	[2  ,   2]  =  -0.848799
	[3  ,   2]  =  0.648284
	[3  ,   3]  =  -0.876228
	[4  ,   3]  =  0.721084
	[4  ,   4]  =  -1.0206
	[5  ,   4]  =  0.816251
	[5  ,   5]  =  -0.86237
	[6  ,   5]  =  0.681295
	⋮
	[195, 195]  =  -0.815069
	[196, 195]  =  0.638504
	[196, 196]  =  -0.814448
	[197, 196]  =  0.825878
	[197, 197]  =  -0.801647
	[198, 197]  =  0.753317
	[198, 198]  =  -0.698868
	[199, 198]  =  0.636046
	[199, 199]  =  -0.703122
	[200, 199]  =  0.643334
	[200, 200]  =  -0.358323

In [33]:
maximum(abs(tril(solShoot-solShootFast)))

1.606411760279869e-7

In [34]:
solShoot[1:7,1:7]

7×7 Array{Float64,2}:
 -0.194102   0.656222   0.0        0.0        0.0        0.0        0.0     
  0.656222  -0.586286   0.671748   0.0        0.0        0.0        0.0     
  0.0        0.671748  -0.695654   0.758151   0.0        0.0        0.0     
  0.0        0.0        0.758151  -0.885963   0.588594   0.0        0.0     
  0.0        0.0        0.0        0.588594  -0.630712   0.633719   0.0     
  0.0        0.0        0.0        0.0        0.633719  -0.767458   0.735599
  0.0        0.0        0.0        0.0        0.0        0.735599  -0.718711